In [15]:
import torch
from transformers import pipeline, AutoTokenizer, AutoModel
import pandas as pd
import json
from tqdm import tqdm
import numpy as np

Helper functions

In [16]:
def batched(lst, batch_size):
    """
    Convert a list of items to a list of batched items with the specified batch size.
    """
    return [lst[i:i+batch_size] for i in range(0, len(lst), batch_size)]

def sigmoid(X):
    return 1 / (1 + np.exp(-X))

def heaviside(X):
    return np.heaviside(X - 0.5, 0)

def f1(preds):
    logits, y_true = preds
    y_pred = heaviside(sigmoid(logits))
    return {'f1':f1_score(y_true, y_pred, average='macro')}


Loading zshot model

In [3]:
model = 'facebook/bart-large-mnli'
task = 'zero-shot-classification'
clf = pipeline(task, model, device=0)
print(clf.device)

cuda:0


Checking zshot model

In [4]:
clf('This is a course about Python Programming', multi_label=True, candidate_labels=['Computer Education', 'Physics Education', 'Computer Networks', 'Python Snake'])

{'sequence': 'This is a course about Python Programming',
 'labels': ['Computer Education',
  'Python Snake',
  'Computer Networks',
  'Physics Education'],
 'scores': [0.8667940497398376,
  0.05108261480927467,
  0.0002302610082551837,
  7.492826989619061e-05]}

Loading train portion of data for 20-class and 54-class z-shot

In [6]:
data_test = pd.read_csv('data/arguments-test.tsv', delimiter='\t')
data_nahj = pd.read_csv('data/arguments-test-nahjalbalagha.tsv', delimiter='\t')
data_nyt = pd.read_csv('data/arguments-test-nyt.tsv', delimiter='\t')

In [7]:
data20 = pd.read_csv('data/prepared/train-zshot-20.tsv', delimiter='\t')
labels20 = list(data20.columns[4:])
data20 = pd.concat([data_test, data_nahj, data_nyt])
# data54 = pd.read_csv('data/prepared/train-zshot-54.tsv', delimiter='\t')
# labels54 = list(data54.columns[4:])

inps = list(data20.apply(lambda x: f'{x.Conclusion} {x.Stance} {x.Premise}', axis=1))

In [11]:
preds20 = []
#preds54 = []
for batch in tqdm(batched(inps, 20)):
    p20 = clf(batch, candidate_labels=labels20, multi_label=True)
    #p54 = clf(batch, candidate_labels=labels54, multi_label=True)

    preds20.append(p20)
    #preds54.append(p54)


  9%|▉         | 9/97 [01:46<17:48, 12.14s/it]/home/erfan/.local/lib/python3.10/site-packages/transformers/pipelines/base.py:1045: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
100%|██████████| 97/97 [22:45<00:00, 14.08s/it]


In [22]:
cols = ['Self-direction: thought','Self-direction: action','Stimulation','Hedonism','Achievement','Power: dominance','Power: resources','Face','Security: personal','Security: societal','Tradition','Conformity: rules','Conformity: interpersonal','Humility','Benevolence: caring','Benevolence: dependability','Universalism: concern','Universalism: nature','Universalism: tolerance','Universalism: objectivity']

In [29]:
len(data_test)

1576

In [32]:
preds_test = []
preds_nahj = []
preds_nyt = []
index = 0
for pbatch in preds20:
    for p in pbatch:
        scores = [int(i) for i in list(heaviside(np.array(p['scores'])))]
        
        if len(preds_test)!=len(data_test):
            preds_test.append([data_test.iloc[index, 0]] + scores)
        elif len(preds_nahj)!=len(data_nahj):
            i = index - len(data_test)
            preds_nahj.append([data_nahj.iloc[i, 0]] + scores)
        else:
            i = index - len(data_test) - len(data_nahj)
            preds_nyt.append([data_nyt.iloc[i, 0]] + scores)
        index += 1

In [40]:
pd.DataFrame(preds_test, columns=['Argument ID']+cols).to_csv(f'experiments/ex_zshot/zshot-test.tsv', sep='\t', index=False)
pd.DataFrame(preds_nahj, columns=['Argument ID']+cols).to_csv(f'experiments/ex_zshot/zshot-nahj.tsv', sep='\t', index=False)
pd.DataFrame(preds_nyt, columns=['Argument ID']+cols).to_csv(f'experiments/ex_zshot/zshot-nyt.tsv', sep='\t', index=False)

In [30]:
# Create a dictionary with the two lists
data = {"preds20": preds20, "preds54": preds54}

# Save the dictionary to a JSON file
with open("data.json", "w") as f:
    json.dump(data, f)
